Dhruv Gandhi and Aayush Dhiman

In [1]:
# https://forums.trakt.tv/t/id-like-an-honest-opinion-which-rating-system-you-trust/1779/2

In [2]:
# /search/trakt/:id?id_type=movie

In [1]:
import obtain_data

In [3]:
obtain_data.main(50)

Fetched 500 trending movies in 4.15 seconds.
Movie certifications and poster URLs fetched/stored successfully in 34.04 seconds.
Movie posters stored successfully in 34.06 seconds.
Fetched usernames of watchers for 500 movies in 79.94 seconds.
Fetched movie ratings for 570 users in 144.53 seconds.


In [4]:
!streamlit run deploy_streamlit.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.110.135.169:8501

2025-04-09 02:19:13.505 Please replace `st.experimental_get_query_params` with `st.query_params`.

`st.experimental_get_query_params` will be removed after 2024-04-11.

Refer to our [docs page](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.query_params) for more information.
2025-04-09 02:19:15.477 Please replace `st.experimental_set_query_params` with `st.query_params`.

`st.experimental_set_query_params` will be removed after 2024-04-11.

Refer to our [docs page](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.query_params) for more information.
2025-04-09 02:19:15.479 Please replace `st.experimental_get_query_params` with `st.query_params`.

`st.experimental_get_query_params` will be removed after 2024-04-11.

Refer to our [docs page](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.quer